mp4 to wav

In [7]:
import os
import glob
import ntpath
import sys

def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

def convert_mp4_to_avi(file_name, output_directory):
    input_name = file_name
    output_name = ntpath.basename(file_name)
    output = os.path.join(output_directory, output_name.replace('.mp4', '.wav', 1))
    cmd = "ffmpeg -i {} -ab 160K -ac 1 -ar 16000 -vn {}".format(input_name, output) 
    return os.popen(cmd)

def main():
    input_directory = '/opt/ml/input/data/videos'
    output_directory = '/opt/ml/input/data/audios'
    createDirectory(output_directory)
    files = glob.glob(input_directory + '/*.mp4')
    for file_name in files:
        convert_mp4_to_avi(file_name, output_directory)
        print('convert mp4 to wav :', file_name)

if __name__ == "__main__":
   main()


convert mp4 to wav : /opt/ml/input/data/videos/Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P.mp4


ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [1]:
%pwd
%cd ..

/opt/ml/input/code/project/torchvggish


In [5]:
from torchvggish import vggish
import numpy as np
import glob

# 변환할 오디오 파일 위치 지정
audio_path = '/opt/ml/input/data/audios'
files = glob.glob(audio_path + '/*.wav')
file_name_list = os.listdir(audio_path)

# 오디오 피처 저장할 위치 지정
feature_save_path = '/opt/ml/input/data/audio_features'

# Initialise model and download weights
model_urls = {
    'vggish': 'https://github.com/harritaylor/torchvggish/'
    'releases/download/v0.1/vggish-10086976.pth',
    'pca': 'https://github.com/harritaylor/torchvggish/'
    'releases/download/v0.1/vggish_pca_params-970ea276.pth'
    }
embedding_model = vggish.VGGish(urls=model_urls, postprocess=False)
embedding_model.eval()

print("[saved list]")
saved_list = []
for audio, audio_name in zip(files, file_name_list):
    embeddings = embedding_model.forward(audio)
    np.save(os.path.join(feature_save_path, audio_name[:-4]),embeddings.detach().cpu().numpy())
    print(os.path.join(feature_save_path, audio_name[:-4])+".npy")
    saved_list.append(os.path.join(feature_save_path, audio_name[:-4])+".npy")

[saved list]
/opt/ml/input/data/audio_features/Bad.Boys.1995__#01-33-51_01-34-37_label_B2-0-0.npy
/opt/ml/input/data/audio_features/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A.npy
/opt/ml/input/data/audio_features/Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P.npy


In [10]:
for sample in saved_list:
    data = np.load(sample)
    print(f'{sample[len(feature_save_path)+1:]} - {data.shape}')

Bad.Boys.1995__#01-33-51_01-34-37_label_B2-0-0.npy - (69, 128)
A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A.npy - (360, 128)
Cold.Eyes_0-00-04_0-00-13_0-00-20_0-00-31_0-01-02_0-01-04_0-01_21_0-01-24_label_P.npy - (139, 128)
